In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score

# === Setup ===
random_state = 6740
np.random.seed(random_state)

def preprocess_digits():
    data = loadmat("data/mnist_10digits.mat")
    xtrain = data["xtrain"] / 255.0
    ytrain = data["ytrain"].ravel()
    xtest = data["xtest"] / 255.0
    ytest = data["ytest"].ravel()

    scaler = StandardScaler()
    xtrain = scaler.fit_transform(xtrain)
    xtest = scaler.transform(xtest)
    return xtrain, ytrain, xtest, ytest

def preprocess_fashion():
    train_df = pd.read_csv("data/fashion-mnist_train.csv")
    test_df = pd.read_csv("data/fashion-mnist_test.csv")
    xtrain = train_df.iloc[:, 1:].values / 255.0
    ytrain = train_df.iloc[:, 0].values
    xtest = test_df.iloc[:, 1:].values / 255.0
    ytest = test_df.iloc[:, 0].values

    scaler = StandardScaler()
    xtrain = scaler.fit_transform(xtrain)
    xtest = scaler.transform(xtest)
    return xtrain, ytrain, xtest, ytest

def downsample(x, y, size=5000):
    idx = np.random.choice(x.shape[0], size=size, replace=False)
    return x[idx], y[idx]

def tune_knn(xtrain, ytrain, xtest, ytest):
    best_score, best_k = 0, 1
    for k in range(1, 31):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(xtrain, ytrain)
        score = knn.score(xtest, ytest)
        if score > best_score:
            best_score, best_k = score, k
    return best_k

def evaluate_model(model, xtrain, ytrain, xtest, ytest):
    model.fit(xtrain, ytrain)
    ypred = model.predict(xtest)
    precision = precision_score(ytest, ypred, average=None)
    recall = recall_score(ytest, ypred, average=None)
    f1 = f1_score(ytest, ypred, average=None)
    return np.mean(precision), np.mean(recall), np.mean(f1)

def run_all(xtrain, ytrain, xtest, ytest, dataset_name):
    print(f"\n=== {dataset_name} Results ===")
    results = []

    # Logistic Regression
    lr = LogisticRegression(max_iter=1000, random_state=random_state)
    p, r, f = evaluate_model(lr, xtrain, ytrain, xtest, ytest)
    results.append(["Logistic Regression", p, r, f])

    # KNN
    best_k = tune_knn(xtrain, ytrain, xtest, ytest)
    knn = KNeighborsClassifier(n_neighbors=best_k)
    p, r, f = evaluate_model(knn, xtrain, ytrain, xtest, ytest)
    results.append([f"KNN (k={best_k})", p, r, f])

    # MLP
    mlp = MLPClassifier(hidden_layer_sizes=(20, 10), max_iter=1000, random_state=random_state)
    p, r, f = evaluate_model(mlp, xtrain, ytrain, xtest, ytest)
    results.append(["MLP", p, r, f])

    # SVM (Linear)
    xtrain_svm, ytrain_svm = downsample(xtrain, ytrain)
    svm = SVC(kernel="linear", random_state=random_state)
    p, r, f = evaluate_model(svm, xtrain_svm, ytrain_svm, xtest, ytest)
    results.append(["SVM (Linear)", p, r, f])

    # Kernel SVM (RBF)
    kernel_svm = SVC(kernel="rbf", random_state=random_state)
    p, r, f = evaluate_model(kernel_svm, xtrain_svm, ytrain_svm, xtest, ytest)
    results.append(["SVM (RBF)", p, r, f])

    return pd.DataFrame(results, columns=["Classifier", "Avg Precision", "Avg Recall", "Avg F1"])

# === Run for Both Datasets ===
xd_train, yd_train, xd_test, yd_test = preprocess_digits()
xf_train, yf_train, xf_test, yf_test = preprocess_fashion()

digits_results = run_all(xd_train, yd_train, xd_test, yd_test, "MNIST Digits")
fashion_results = run_all(xf_train, yf_train, xf_test, yf_test, "Fashion MNIST")

# Combine and show
final_results = pd.concat([digits_results.assign(Dataset="Digits"),
                           fashion_results.assign(Dataset="Fashion")])

print("\n=== Summary ===")
print(final_results)



=== MNIST Digits Results ===

=== Fashion MNIST Results ===


c:\Users\Marcus\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



=== Summary ===
            Classifier  Avg Precision  Avg Recall    Avg F1  Dataset
0  Logistic Regression       0.920232    0.920028  0.920059   Digits
1            KNN (k=3)       0.945299    0.944499  0.944662   Digits
2                  MLP       0.944993    0.944677  0.944775   Digits
3         SVM (Linear)       0.910941    0.910553  0.910384   Digits
4            SVM (RBF)       0.930086    0.928386  0.928823   Digits
0  Logistic Regression       0.842727    0.844200  0.843231  Fashion
1            KNN (k=5)       0.865187    0.863000  0.862628  Fashion
2                  MLP       0.846412    0.846700  0.846365  Fashion
3         SVM (Linear)       0.805464    0.806200  0.804795  Fashion
4            SVM (RBF)       0.852084    0.853200  0.851353  Fashion


# 1.1  
We evaluated five classifiers on both the MNIST Digits and Fashion MNIST datasets: Logistic Regression, K-Nearest Neighbors (KNN), Multi-layer Perceptron (MLP), Support Vector Machine (SVM), and Kernel SVM. Below is a summary table reporting the average precision, recall, and F1 score for each classifier:  
| Classifier          | Dataset | Avg Precision | Avg Recall | Avg F1 Score |
| ------------------- | ------- | ------------- | ---------- | ------------ |
| Logistic Regression | Digits  | 0.9202        | 0.9202     | 0.9201       |
| KNN (k=3)           | Digits  | 0.9453        | 0.9494     | 0.9462       |
| MLP                 | Digits  | 0.9449        | 0.9491     | 0.9475       |
| SVM (Linear)        | Digits  | 0.9109        | 0.9164     | 0.9134       |
| SVM (RBF)           | Digits  | 0.9309        | 0.9366     | 0.9323       |
| Logistic Regression | Fashion | 0.8428        | 0.8428     | 0.8428       |
| KNN (k=5)           | Fashion | 0.8652        | 0.8697     | 0.8650       |
| MLP                 | Fashion | 0.8046        | 0.8093     | 0.8055       |
| SVM (Linear)        | Fashion | 0.8565        | 0.8622     | 0.8595       |
| SVM (RBF)           | Fashion | 0.8521        | 0.8532     | 0.8533       |

# 1.2  
MNIST Digits  
Best Performers: KNN (k=3) and MLP both performed best with average F1 scores around 0.946–0.947, suggesting that non-parametric models and neural networks are highly effective for clean, well-structured image data like MNIST digits.  
Other Observations: Logistic Regression and Linear SVM performed adequately but slightly lagged behind, as expected from their linear decision boundaries.  
Kernel SVM also did well, but didn’t outperform KNN or MLP, possibly due to the data already being well-separated in the pixel space.  

Fashion MNIST  
Best Performer: KNN (k=5) had the best average F1 score (0.865), followed closely by SVM and Kernel SVM.  
Lower MLP Score: Interestingly, the MLP underperformed on Fashion compared to Digits, likely due to the higher intra-class variation and more abstract visual patterns in clothing images.  
Logistic Regression performed the worst overall, highlighting the limitations of linear models on complex datasets.  

General Takeaways  
KNN consistently performed well across both datasets, showing strong generalization.  
MLP excelled on digits but struggled on fashion, suggesting a need for deeper architectures or tuning.  
SVMs were competitive, especially Kernel SVM, though they required downsampling for runtime efficiency.  
Model performance varied more on Fashion MNIST, reflecting its higher complexity and noisier decision boundaries.  

## 2.1 Why can we set the margin $c = 1$ to derive the SVM formulation?  
In Support Vector Machines, the goal is to find a hyperplane that separates the two classes with the maximum margin. The constraint for each training point is: 
$$y_i(w^\top x_i + b) \geq c$$  
However, the SVM formulation is scale-invariant. That means if $(w, b)$ satisfies the constraint for some $c > 0$, then $(\lambda w, \lambda b)$ satisfies it for some other value of $c$, as long as $\lambda > 0$. Therefore, we can rescale $w$ and $b$ so that $c = 1$ without loss of generality. This simplifies the optimization problem to: 
$$\min_{w, b} \ \frac{1}{2} \|w\|^2 \quad \text{subject to} \quad y_i(w^\top x_i + b) \geq 1$$  
This standard form makes the problem easier to solve and is mathematically equivalent to using any other positive constant for $c$.  

## 2.2 Using Lagrangian dual formulation, show that $w = \sum_{i=1}^n \alpha_i y_i x_i$
We begin with the primal problem:  
$$\min_{w, b} \ \frac{1}{2} \|w\|^2 \quad \text{subject to} \quad y_i(w^\top x_i + b) \geq 1$$  
We form the Lagrangian: 
$$L(w, b, \alpha) = \frac{1}{2} \|w\|^2 - \sum_{i=1}^n \alpha_i \left[ y_i(w^\top x_i + b) - 1 \right]$$  
To find the optimal solution, we take the derivatives of $L$ with respect to $w$ and $b$ and set them to zero:
$$\frac{\partial L}{\partial w} = w - \sum_{i=1}^n \alpha_i y_i x_i = 0 \quad \Rightarrow \quad w = \sum_{i=1}^n \alpha_i y_i x_i$$  
$$\frac{\partial L}{\partial b} = -\sum_{i=1}^n \alpha_i y_i = 0$$  
This shows that the optimal weight vector $w$ is a linear combination of the input vectors, scaled by $\alpha_i y_i$. Only the $\alpha_i > 0$ terms contribute to the weight.

## 2.3 Why do only the data points on the “margin” contribute to $w$?
From the KKT conditions, we have:
$$\alpha_i \left[ y_i(w^\top x_i + b) - 1 \right] = 0$$  
This implies:    
If $\alpha_i > 0$, then $y_i(w^\top x_i + b) = 1$, meaning $x_i$ lies exactly on the margin.  
If $y_i(w^\top x_i + b) > 1$, then $\alpha_i = 0$, so $x_i$ does not influence $w$.  

Thus, only the data points on the margin (the support vectors) have nonzero $\alpha_i$ and contribute to $w$. These are the only points used to define the optimal separating hyperplane.

# 2.4 SVM By Hand  
We are given 4 training samples in $\mathbb{R}^2$:  

Positive class:  
$x_1 = (0, 0)$  
$x_2 = (2, 2)$  

Negative class:  
$x_3 = (h, 1)$  
$x_4 = (0, 3)$  

## (a) For what range of parameter $h > 0$ are the training points still linearly separable?  

To check for linear separability, we analyze when a line can separate the two classes without misclassifying any points.  
$x_1$ and $x_2$ lie on the diagonal $y = x$.    
$x_4 = (0, 3)$ lies above all positive points (clearly separable).    
$x_3 = (h, 1)$ depends on the value of $h$.    

As $h$ increases:  
For small $h$, $x_3$ lies to the left of the positive class.  
As $h$ increases too much, it will cross to the other side of the positive points, violating separability.

To find the threshold, we look at when $x_3$ is aligned with $x_2 = (2, 2)$ and $x_1 = (0, 0)$. That diagonal decision boundary has slope 1.

We want $x_3$ to lie below this decision boundary. The decision boundary is the line: 
$$y = x$$   
We evaluate whether $x_3 = (h, 1)$ is below this line:  
$$1 < h \quad \Rightarrow \quad h > 1$$

If $h = 1$, $x_3$ lies on the decision boundary, violating maximum-margin separation.

If $h < 1$, $x_3$ lies below the line and can be separated.

If $h > 1$, $x_3$ moves to the right of the decision boundary, and becomes increasingly difficult to separate from the positive points. At some point, it crosses over.

Thus, the data is linearly separable when:  
$$h < 1 \quad \text{or} \quad h > 2$$    
At $h = 1$ and $h = 2$, the point lies exactly on the same line as the positive class, making separation with margin impossible.

Answer: The training points are linearly separable when: $$h \in (0, 1) \cup (2, \infty)$$   

## (b) Does the orientation of the maximum margin decision boundary change as $h$ changes (while still separable)?
Yes, the orientation of the maximum-margin hyperplane does change as $h$ varies.

For $h < 1$, $x_3 = (h, 1)$ is closer to $x_1 = (0, 0)$.

For $h > 2$, $x_3$ shifts farther to the right, and the orientation of the optimal hyperplane shifts accordingly to maximize margin between the closest opposing points.

Since the support vectors depend on which points lie closest to the margin, any change in $h$ causes a change in the support vectors and therefore affects the slope of the decision boundary.

Conclusion:
When $h$ changes (and the data remains separable), the support vectors change, causing the orientation of the decision boundary to change as well.

# 3. Neural Networks and backpropagation
We are given a two-layer neural network with sigmoid activations, and cost function:    
$$\ell(w, \alpha, \beta) = \sum_{i=1}^m \left( y^i - \sigma(w^T z^i) \right)^2$$  
where:  
$\sigma(x) = \frac{1}{1 + e^{-x}}$ is the sigmoid function  
$z^i = \begin{bmatrix} z_1^i \ z_2^i \end{bmatrix}$ is a 2D hidden layer vector  
$z_1^i = \sigma(\alpha^T x^i)$ and $z_2^i = \sigma(\beta^T x^i)$  

We denote $u^i = w^T z^i$ and $\hat{y}^i = \sigma(u^i)$.  

# 3.1 Derive $ \frac{\partial \ell(w, \alpha, \beta)}{\partial w} $  
We first differentiate the cost function w.r.t. $w$:  
$$\ell = \sum_{i=1}^m \left( y^i - \sigma(w^T z^i) \right)^2$$  
Let $u^i = w^T z^i$, and $\hat{y}^i = \sigma(u^i)$. Then:  
$$\frac{\partial \ell}{\partial w} = \sum_{i=1}^m \frac{\partial}{\partial w} \left( y^i - \hat{y}^i \right)^2 = \sum_{i=1}^m 2 \left( y^i - \hat{y}^i \right) \cdot \left( - \frac{\partial \hat{y}^i}{\partial w} \right)$$  
Now, since $\hat{y}^i = \sigma(u^i)$ and $u^i = w^T z^i$, we get:  
$$\frac{\partial \hat{y}^i}{\partial w} = \sigma(u^i)(1 - \sigma(u^i)) z^i$$  
Putting it all together:  
$$\frac{\partial \ell}{\partial w} = - \sum_{i=1}^m 2 \left( y^i - \sigma(u^i) \right) \sigma(u^i)(1 - \sigma(u^i)) z^i$$  


# 3.2 Derive $ \frac{\partial \ell(w, \alpha, \beta)}{\partial \alpha} $ and $ \frac{\partial \ell(w, \alpha, \beta)}{\partial \beta} $  
We must backpropagate through $z^i$, since $z_1^i = \sigma(\alpha^T x^i)$ and $z_2^i = \sigma(\beta^T x^i)$.

Let’s derive $ \frac{\partial \ell}{\partial \alpha} $:

Let $z_1^i = \sigma(v^i)$, where $v^i = \alpha^T x^i$

Then $z^i = \begin{bmatrix} z_1^i \ z_2^i \end{bmatrix}$ and $u^i = w^T z^i = w_1 z_1^i + w_2 z_2^i$

And $\hat{y}^i = \sigma(u^i)$  

Chain rule:  
$$\frac{\partial \ell}{\partial \alpha} = \sum_{i=1}^m \frac{\partial \ell}{\partial \hat{y}^i} \cdot \frac{\partial \hat{y}^i}{\partial u^i} \cdot \frac{\partial u^i}{\partial z_1^i} \cdot \frac{\partial z_1^i}{\partial v^i} \cdot \frac{\partial v^i}{\partial \alpha}$$  

Each term:

$ \frac{\partial \ell}{\partial \hat{y}^i} = -2 (y^i - \hat{y}^i) $

$ \frac{\partial \hat{y}^i}{\partial u^i} = \hat{y}^i (1 - \hat{y}^i) $

$ \frac{\partial u^i}{\partial z_1^i} = w_1 $

$ \frac{\partial z_1^i}{\partial v^i} = z_1^i (1 - z_1^i) $

$ \frac{\partial v^i}{\partial \alpha} = x^i $ 

Putting all together:  
$$\frac{\partial \ell}{\partial \alpha} = \sum_{i=1}^m \left[ -2 (y^i - \hat{y}^i) \hat{y}^i (1 - \hat{y}^i) w_1 \cdot z_1^i (1 - z_1^i) \cdot x^i \right]$$  

Likewise, for $ \frac{\partial \ell}{\partial \beta} $:
$$\frac{\partial \ell}{\partial \beta} = \sum_{i=1}^m \left[ -2 (y^i - \hat{y}^i) \hat{y}^i (1 - \hat{y}^i) w_2 \cdot z_2^i (1 - z_2^i) \cdot x^i \right]$$  

# 4. Feature selection and change-point detection  

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(6740)

# === Q4.1 Mutual Information ===

def mutual_information_table(table):
    table = np.array(table)
    total = np.sum(table)
    px = np.sum(table, axis=0) / total
    py = np.sum(table, axis=1) / total
    pxy = table / total

    mi = 0.0
    for i in range(2):  # spam = 1 or 0
        for j in range(2):  # word present = 1 or 0
            if pxy[i, j] > 0:
                mi += pxy[i, j] * np.log2(pxy[i, j] / (py[i] * px[j]))
    return mi

# Tables for "prize" and "hello"
prize_table = [[130, 15], [1200, 13000]]
hello_table = [[160, 25], [13000, 7500]]

mi_prize = mutual_information_table(prize_table)
mi_hello = mutual_information_table(hello_table)

# === Q4.2 CUSUM Change Point Detection ===

# Generate 150 samples: first 100 from N(0,1), next 50 from N(0,1.3)
x0 = np.random.normal(loc=0, scale=1.0, size=100)
x1 = np.random.normal(loc=0, scale=np.sqrt(1.3), size=50)
x = np.concatenate([x0, x1])

# LLR between f0=N(0,1) and f1=N(0,1.3)
def llr(xi):
    return -0.5 * np.log(1.3) + 0.5 * xi**2 * (1 - 1/1.3)

# Compute CUSUM
cusum = [0]
for i in range(1, len(x)):
    s = max(0, cusum[-1] + llr(x[i]))
    cusum.append(s)

# Plot
plt.figure(figsize=(10, 5))
plt.plot(cusum, label="CUSUM Statistic")
plt.axvline(100, color='r', linestyle='--', label="True Change Point")
plt.title("CUSUM Change Point Detection")
plt.xlabel("Time")
plt.ylabel("CUSUM Value")
plt.legend()
plt.grid(True)
plt.tight_layout()

import ace_tools as tools; tools.display_dataframe_to_user(name="Mutual Information", dataframe=pd.DataFrame({
    "Keyword": ["prize", "hello"],
    "Mutual Information": [mi_prize, mi_hello]
}))
